#  Practice Optimizing DataFrames and Processing in Chunks

## Introduction

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 99

In [2]:
df = pd.read_csv('loans_2007.csv')
print(df.head())

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade                 emp_title emp_length  \
0   10.65%       162.87     B        B2                       NaN  10+ years   
1   15.27%        59.83     C        C4                     Ryder   < 1 year   
2   15.96%        84.33     C        C5                       NaN  10+ years   
3   13.49%       339.31     C        C1       AIR RESOURCES BOARD  10+ years   
4   12.69%        67.79     B        B5  University Medical Group     1 year   

  home_ownership  annual_inc verification_status  

/tmp/ipykernel_841/561780463.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('loans_2007.csv')


In [3]:
thousand_row = pd.read_csv('loans_2007.csv', nrows=1000)
thousand_row.memory_usage(deep=True).sum()/(1024*1024)

np.float64(1.5273666381835938)

### Let's try tripling to 3000 rows and calculate the memory footprint for each chunk.

In [4]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in chunk_iter:
    print(chunk.memory_usage(deep=True).sum()/(1024*1024))

4.580394744873047
4.576141357421875
4.577898979187012
4.579251289367676
4.575444221496582
4.577326774597168
4.575918197631836
4.578287124633789
4.576413154602051
4.57646369934082
4.589176177978516
4.588043212890625
4.594850540161133
4.828314781188965
0.868586540222168


### how many rows in the dataset?

In [5]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
total_rows = 0
for chunk in chunk_iter:
    total_rows += len(chunk)
print(total_rows)

42538


## Exploring the Data in Chunks

### How many columns have a numeric type? How many have a string type?

In [6]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
num_cols = []
str_cols = []
for chunk in chunk_iter:
    num = chunk.select_dtypes(include=[np.number]).shape[1]
    num_cols.append(num)
    string = chunk.select_dtypes(include=['object']).shape[1]
    str_cols.append(string)
print(num_cols)
print(str_cols)

[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


In [7]:
# check if string cols are consistent in each chunk
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
obj_col = []
for chunk in chunk_iter:
    chunk_obj_col = chunk.select_dtypes(include=['object']).columns.to_list()
    if len(obj_col) > 0:
        is_same = chunk_obj_col == obj_col
        if not is_same:
            print('overall string cols are:', obj_col)
            print('chunk string cols are:', chunk_obj_col)
            
    else:
        obj_col = chunk_obj_col

overall string cols are: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']
chunk string cols are: ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']
overall string cols are: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'appl

Observation 1: By default — 31 numeric columns and 21 string columns.

Observation 2: It seems like one column in particular (the id column) is being cast to int64 in the last 2 chunks but not in the earlier chunks. Since the id column won't be useful for analysis, visualization, or predictive modeling, let's ignore this column.

### How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?

In [8]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
unique = {}
total_rows = 0
for chunk in chunk_iter:
    se_cols = chunk.select_dtypes(include=['object'])
    fin_cols = se_cols.columns
    total_rows += len(chunk)
    for col in fin_cols:
        val_count = se_cols[col].value_counts()
        if col in unique:
            unique[col].append(val_count)
        else:
            unique[col] = [val_count]
            
col_les_50_perc = 0 
col_uni_val = {}
for c in unique:
    concat_col = pd.concat(unique[c])
    group_count = concat_col.groupby(concat_col.index).sum()
    col_uni_val[c] = len(group_count)
    uni_perc = len(group_count)/total_rows
    if uni_perc < 0.5:
        col_les_50_perc += 1    
print(col_uni_val)
print(col_les_50_perc)
            
    

{'term': 2, 'int_rate': 394, 'grade': 7, 'sub_grade': 35, 'emp_title': 30658, 'emp_length': 11, 'home_ownership': 5, 'verification_status': 3, 'issue_d': 55, 'loan_status': 9, 'pymnt_plan': 2, 'purpose': 14, 'title': 21264, 'zip_code': 837, 'addr_state': 50, 'earliest_cr_line': 530, 'revol_util': 1119, 'initial_list_status': 1, 'last_pymnt_d': 103, 'last_credit_pull_d': 108, 'application_type': 1, 'id': 3538}
21


In [9]:
# another way to use set instead of dictionary to do unique values counts

chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
uniques_per_col = {}
total_rows = 0
for chunk in chunk_iter:
    se_cols = chunk.select_dtypes(include=['object'])
    col_names = se_cols.columns
    total_rows += len(chunk)
    
    for col in col_names:
        
        val_count = se_cols[col].value_counts()
        if col in uniques_per_col:
            uniques_per_col[col].update(val_count.index)
        else:
            uniques_per_col[col] = set(val_count.index)
            
# create a dictionary for the amount of unique values of each column             
num_uni_col = {}
for key in uniques_per_col.keys():
    num_uni_col[key] = len(uniques_per_col[key])
print(num_uni_col)

num_cols_less_50_perc = 0
for val in num_uni_col.values():
    if val/total_rows < 0.5:
        num_cols_less_50_perc += 1
    
print(total_rows)
print(num_cols_less_50_perc)

{'term': 2, 'int_rate': 394, 'grade': 7, 'sub_grade': 35, 'emp_title': 30658, 'emp_length': 11, 'home_ownership': 5, 'verification_status': 3, 'issue_d': 55, 'loan_status': 9, 'pymnt_plan': 2, 'purpose': 14, 'title': 21264, 'zip_code': 837, 'addr_state': 50, 'earliest_cr_line': 530, 'revol_util': 1119, 'initial_list_status': 1, 'last_pymnt_d': 103, 'last_credit_pull_d': 108, 'application_type': 1, 'id': 3538}
42538
21


In [10]:
df = pd.read_csv('loans_2007.csv')
ob_cols = df.select_dtypes(include=['object'])
print(len(ob_cols.columns))

22


/tmp/ipykernel_841/1169366068.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('loans_2007.csv')


In [11]:
col_uni = {}
for col in ob_cols.columns:
    col_uni[col] = ob_cols[col].nunique()
print(col_uni)
    

{'id': 42538, 'term': 2, 'int_rate': 394, 'grade': 7, 'sub_grade': 35, 'emp_title': 30658, 'emp_length': 11, 'home_ownership': 5, 'verification_status': 3, 'issue_d': 55, 'loan_status': 9, 'pymnt_plan': 2, 'purpose': 14, 'title': 21264, 'zip_code': 837, 'addr_state': 50, 'earliest_cr_line': 530, 'revol_util': 1119, 'initial_list_status': 1, 'last_pymnt_d': 103, 'last_credit_pull_d': 108, 'application_type': 1}


### Which float columns have no missing values and could be candidates for conversion to the integer type?

In [12]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
missing = []
for chunk in chunk_iter:
    floats_col_df = chunk.select_dtypes(include=['float'])
    missing.append(floats_col_df.apply(pd.isnull).sum())
    
combined_missing = pd.concat(missing)
col_mis_val_amount = combined_missing.groupby(combined_missing.index).sum().sort_values()
print(col_mis_val_amount)

collection_recovery_fee          3
dti                              3
loan_amnt                        3
member_id                        3
last_pymnt_amnt                  3
installment                      3
funded_amnt_inv                  3
funded_amnt                      3
total_pymnt                      3
total_pymnt_inv                  3
total_rec_int                    3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_prncp                  3
total_rec_late_fee               3
annual_inc                       7
open_acc                        32
delinq_amnt                     32
delinq_2yrs                     32
acc_now_delinq                  32
inq_last_6mths                  32
pub_rec                         32
total_acc                       32
tax_liens                      108
collections_12_mths_ex_med     148
chargeoff_within_12_

## Calculate the total memory usage across all of the chunks.

In [13]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
total_mem_use = []
for chunk in chunk_iter:
    total_mem_use.append(chunk.memory_usage(deep=True).sum()/(1024*1024))
print(sum(total_mem_use))

65.24251079559326


In [14]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
total_mem_use = 0
for chunk in chunk_iter:
    total_mem_use += chunk.memory_usage(deep=True).sum()/(1024*1024)
print(total_mem_use)

65.24251079559326


# Optimizing String Columns

## Determine which string columns you can convert to a numeric type if you clean them. Let's focus on columns that would actually be useful for analysis and modeling.


In [15]:
obj_col 


['term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type']

In [16]:
use_ob_cols = ['term', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'issue_d', 'purpose', 'earliest_cr_line', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d']

In [17]:
chunk_loan = pd.read_csv('loans_2007.csv', chunksize=3000)
loan_unique_val_counts = {}
for chunk in chunk_loan:
    str_cols = chunk.select_dtypes(include=['object'])
    for col in str_cols.columns:
        val_count = str_cols[col].value_counts()
        if col in loan_unique_val_counts:
            loan_unique_val_counts[col].append(val_count)
        else:
            loan_unique_val_counts[col] = [val_count]
            
combined_unique_count = {}
for col in loan_unique_val_counts:
    combined_col = pd.concat(loan_unique_val_counts[col])
    final_val_counts = combined_col.groupby(combined_col.index).sum()
    combined_unique_count[col] = final_val_counts
#print(combined_unique_count)
print(combined_unique_count.keys())   

dict_keys(['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type', 'id'])


In [18]:
for colu in use_ob_cols:
    print(colu)
    print(combined_unique_count[colu])
    print("-----------")
    

term
term
36 months    31534
60 months    11001
Name: count, dtype: int64
-----------
sub_grade
sub_grade
A1    1142
A2    1520
A3    1823
A4    2905
A5    2793
B1    1882
B2    2113
B3    2997
B4    2590
B5    2807
C1    2264
C2    2157
C3    1658
C4    1370
C5    1291
D1    1053
D2    1485
D3    1322
D4    1140
D5    1016
E1     884
E2     791
E3     668
E4     552
E5     499
F1     392
F2     308
F3     236
F4     211
F5     154
G1     141
G2     107
G3      79
G4      99
G5      86
Name: count, dtype: int64
-----------
emp_title
emp_title
  old palm inc                       1
 Brocade Communications              1
 CenturyLink                         1
 Department of Homeland Security     1
 Down To Earth Distributors, Inc.    1
                                    ..
zashko inc.                          1
zeno office solutions                1
zion lutheran school                 1
zoll medical corp                    1
zozaya officiating                   1
Name: count, Length: 3

In [19]:
# chunk_loan = pd.read_csv('loans_2007.csv', chunksize=3000)
# loan_unique_val_counts = {}
# for chunk in chunk_loan:
#     str_cols = chunk.select_dtypes(include=['object'])
#     for col in str_cols.columns:
#         val_count = str_cols[col].value_counts()
#         if col in loan_unique_val_counts:
#             loan_unique_val_counts[col].update(val_count.index)
#         else:
#             loan_unique_val_counts[col] = set(val_count.index)
            
# combined_unique_count = {}
# for col in loan_unique_val_counts:
#     combined_col = pd.concat(loan_unique_val_counts[col])
#     final_val_counts = combined_col.groupby(combined_col.index).sum()
#     combined_unique_count[col] = final_val_counts
# print(combined_unique_count)

### Convert to category.

In [20]:
convert_col_dtype = {'sub_grade':'category', 'home_ownership':'category', 'verification_status':'category', 'purpose':'category'}

### Convert term and revol_util to numerical by data cleaning.


### Convert issue_d, earliest_cr_line, last_pymnt_d, and last_credit_pull_d to datetime.


In [21]:
chunk[use_ob_cols]

,term,sub_grade,emp_title,home_ownership,verification_status,issue_d,purpose,earliest_cr_line,revol_util,last_pymnt_d,last_credit_pull_d
42000,36 months,C2,Best Buy,RENT,Not Verified,Feb-2008,debt_consolidation,Jul-2000,100.7%,Feb-2011,Jun-2016
42001,36 months,G2,CVS PHARMACY,OWN,Not Verified,Feb-2008,debt_consolidation,Mar-1989,51.9%,Nov-2008,Jun-2016
42002,36 months,E4,General Motors,RENT,Not Verified,Feb-2008,debt_consolidation,Dec-1998,80.7%,Feb-2011,Jun-2016
42003,36 months,G4,usa medical center,RENT,Not Verified,Feb-2008,debt_consolidation,Jul-1995,57.2%,Feb-2011,Jun-2011
42004,36 months,B3,InvestSource Inc,RENT,Not Verified,Feb-2008,debt_consolidation,Sep-2005,74%,Mar-2010,Aug-2010
...,...,...,...,...,...,...,...,...,...,...,...
42533,36 months,B3,NaN,RENT,Not Verified,Jun-2007,other,NaN,NaN,Jun-2010,May-2007
42534,36 months,A5,NaN,NONE,Not Verified,Jun-2007,other,NaN,NaN,Jun-2010,Aug-2007
42535,36 months,A3,Homemaker,MORTGAGE,Not Verified,Jun-2007,other,NaN,NaN,Jun-2010,Feb-2015
42536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
##  Calculate missing values

In [23]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000, dtype=convert_col_dtype, parse_dates=['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d'])
float_cols_count = {}
mem_use = 0
for chunk in chunk_iter:
        term_clean = chunk['term'].apply(lambda x: str(x).lstrip(' ').rstrip(' months') if not pd.isna(x) else x)
        revol_clean = chunk['revol_util'].apply(lambda x: str(x).rstrip('%') if not pd.isna(x) else x)
        chunk['term'] = pd.to_numeric(term_clean)
        chunk['revol_util'] = pd.to_numeric(revol_clean)
        float_cols = chunk.select_dtypes(include=['float'])
        mem_use  += chunk.memory_usage(deep=True).sum()/(1024**2
                                                          )
        for col in float_cols.columns:
            missing_values = int(len(chunk) - float_cols[col].count())
            if col in float_cols_count:
                float_cols_count[col] += missing_values
            else:
                float_cols_count[col] = missing_values
                
print(float_cols_count)
print(mem_use)

# another way to process the data 
#     term_cleaned = chunk['term'].str.lstrip(" ").str.rstrip(" months")
#     revol_cleaned = chunk['revol_util'].str.rstrip("%")
#     chunk['term'] = pd.to_numeric(term_cleaned)
#     chunk['revol_util'] = pd.to_numeric(revol_cleaned)   

/tmp/ipykernel_841/1026666455.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunk_iter:
/tmp/ipykernel_841/1026666455.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunk_iter:
/tmp/ipykernel_841/1026666455.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunk_iter:
/tmp/ipykernel_841/1026666455.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunk_iter:
/tmp/ipykern

{'member_id': 3, 'loan_amnt': 3, 'funded_amnt': 3, 'funded_amnt_inv': 3, 'installment': 3, 'annual_inc': 7, 'dti': 3, 'delinq_2yrs': 32, 'inq_last_6mths': 32, 'open_acc': 32, 'pub_rec': 32, 'revol_bal': 3, 'revol_util': 93, 'total_acc': 32, 'out_prncp': 3, 'out_prncp_inv': 3, 'total_pymnt': 3, 'total_pymnt_inv': 3, 'total_rec_prncp': 3, 'total_rec_int': 3, 'total_rec_late_fee': 3, 'recoveries': 3, 'collection_recovery_fee': 3, 'last_pymnt_amnt': 3, 'collections_12_mths_ex_med': 148, 'policy_code': 3, 'acc_now_delinq': 32, 'chargeoff_within_12_mths': 148, 'delinq_amnt': 32, 'pub_rec_bankruptcies': 1368, 'tax_liens': 108, 'term': 3}
41.08497142791748


/tmp/ipykernel_841/1026666455.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunk_iter:
/tmp/ipykernel_841/1026666455.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunk_iter:
/tmp/ipykernel_841/1026666455.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunk_iter:
/tmp/ipykernel_841/1026666455.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  for chunk in chunk_iter:
/tmp/ipykern